In [ ]:
import os
import numpy as np
import math
import itertools
import sys
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import json
import pandas as pd
from PIL import Image

from models import get_instance_segmentation_model,train_one_epoch, _get_iou_types
import utils
from datasets import get_gatitos_df,get_transform, GatitosDataset, CocoDataset, get_df_from_coco

from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator
from utils import precision_recall, get_precisionDF
import tqdm

In [ ]:
import torch


In [ ]:
# select categories 
#path="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/annotations/instances_train2014.json"
path="D:/datos/annotations_trainval2017/annotations/instances_train2017.json"
with open(path) as json_file:
    data = json.load(json_file)

dfCategories=pd.DataFrame(data['categories'])
dfCategories[dfCategories['supercategory']=='animal']

In [4]:
dfCategories[dfCategories['supercategory']=='appliance']

,supercategory,id,name
68,appliance,78,microwave
69,appliance,79,oven
70,appliance,80,toaster
71,appliance,81,sink
72,appliance,82,refrigerator


In [5]:
dfCategories[dfCategories['supercategory']=='person']

,supercategory,id,name
0,person,1,person


In [6]:
set(dfCategories['supercategory'])

{'accessory',
 'animal',
 'appliance',
 'electronic',
 'food',
 'furniture',
 'indoor',
 'kitchen',
 'outdoor',
 'person',
 'sports',
 'vehicle'}

In [7]:
dfCategories['supercategory'].value_counts()

food          10
sports        10
animal        10
vehicle        8
kitchen        7
indoor         7
electronic     6
furniture      6
appliance      5
outdoor        5
accessory      5
person         1
Name: supercategory, dtype: int64

In [8]:
#40719
#df=get_df_from_coco(training_path_json,categories)

In [9]:
torch.cuda.is_available()

True

In [10]:
num_epochs = 25
epoch_ini=3
#training_path_json="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/annotations/instances_train2014.json"
training_path_json="D:/datos/annotations_trainval2017/annotations/instances_train2017.json"
#training_path_images="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/images/train2014/"
training_path_images="D:/datos/train2017/train2017/"
validation_path_json="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/annotations/instances_val2014.json"
validation_path_images="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/images/val2014/"

categories=[1]#17,18]
batch_size=3
n_cpu=4
# our dataset has two classes only - background and person
num_classes = len(categories)+1

In [11]:
#device = torch.device('cuda') if False else torch.device('cpu')
#device

In [12]:
os.makedirs("saved_models", exist_ok=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

In [ ]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001,#lr 0.005 y momentum a 0.9 entrenar cats y gogs
                            momentum=0.9, weight_decay=0.0005)#


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
print(f"epoch_ini= {epoch_ini}")
if epoch_ini != 0:
    # Load pretrained models
    
    model.load_state_dict(torch.load("saved_models/generator_%d.pth" % epoch_ini))
    epoch_ini=epoch_ini+1
    print("loaded")

In [ ]:
#CocoDataset(path_json=training_path_json,path_images=training_path_images,
#                categories=categories,transforms=get_transform(train=True))

In [13]:
train_dataloader = torch.utils.data.DataLoader(
    CocoDataset(path_json=training_path_json,path_images=training_path_images,
                categories=categories,transforms=get_transform(train=True))
    , batch_size=batch_size, shuffle=True, num_workers=n_cpu,
    collate_fn=utils.collate_fn)






3888
4154
7917
8314
8486
8547
14471
14528
14613
14810
area por debajo en de 10 pixeles en 14901
14923
15378
area por debajo en de 10 pixeles en 20821
area por debajo en de 10 pixeles en 20964
area por debajo en de 10 pixeles en 21017
area por debajo en de 10 pixeles en 21091
21241
21261
21362
area por debajo en de 10 pixeles en 21466
21482
21521
area por debajo en de 10 pixeles en 21600
area por debajo en de 10 pixeles en 21675
area por debajo en de 10 pixeles en 21788
area por debajo en de 10 pixeles en 25836
area por debajo en de 10 pixeles en 26017
area por debajo en de 10 pixeles en 26041
area por debajo en de 10 pixeles en 26078
area por debajo en de 10 pixeles en 26085
area por debajo en de 10 pixeles en 27619
area por debajo en de 10 pixeles en 27624
area por debajo en de 10 pixeles en 27679
27706
area por debajo en de 10 pixeles en 27867
28335
33845
34787
35915
40371
area por debajo en de 10 pixeles en 43792
area por debajo en de 10 pixeles en 44303
area por debajo en de 10 pix

area por debajo en de 10 pixeles en 547313
547348
area por debajo en de 10 pixeles en 547409
area por debajo en de 10 pixeles en 547479
area por debajo en de 10 pixeles en 547662
area por debajo en de 10 pixeles en 547769
550009
area por debajo en de 10 pixeles en 550127
550326
area por debajo en de 10 pixeles en 550376
550662
550786
area por debajo en de 10 pixeles en 551288
551385
551459
551604
551721
551919
551940
552472
553131
553194
553230
area por debajo en de 10 pixeles en 553301
553543
553781
area por debajo en de 10 pixeles en 553904
554244
area por debajo en de 10 pixeles en 554537
554633
554945
555001
555585
area por debajo en de 10 pixeles en 556176
area por debajo en de 10 pixeles en 556182
area por debajo en de 10 pixeles en 556378
556384
556634
area por debajo en de 10 pixeles en 556703
557780
557975
558330
area por debajo en de 10 pixeles en 559114
559261
559648
area por debajo en de 10 pixeles en 559921
area por debajo en de 10 pixeles en 559959
559975
560001
area por 

In [14]:
validation_dataloader = torch.utils.data.DataLoader(
    CocoDataset(path_json=validation_path_json,path_images=validation_path_images,
                categories=categories,transforms=get_transform(train=False))
    , batch_size=batch_size, shuffle=True, num_workers=n_cpu,
    collate_fn=utils.collate_fn)

4476
4769
8898
9121
9576
9767
9836
16548
16616
16712
16943
area por debajo en de 10 pixeles en 17052
17081
17520
17596
area por debajo en de 10 pixeles en 23827
area por debajo en de 10 pixeles en 23998
area por debajo en de 10 pixeles en 24058
area por debajo en de 10 pixeles en 24059
area por debajo en de 10 pixeles en 24144
24175
24308
24330
24333
24451
area por debajo en de 10 pixeles en 24572
24590
24627
24632
area por debajo en de 10 pixeles en 24733
24810
area por debajo en de 10 pixeles en 24820
area por debajo en de 10 pixeles en 24947
area por debajo en de 10 pixeles en 29511
area por debajo en de 10 pixeles en 29712
area por debajo en de 10 pixeles en 29740
area por debajo en de 10 pixeles en 29781
area por debajo en de 10 pixeles en 29789
area por debajo en de 10 pixeles en 31543
area por debajo en de 10 pixeles en 31548
31611
area por debajo en de 10 pixeles en 31620
31653
area por debajo en de 10 pixeles en 31839
32327
32374
38619
39686
40287
41008
45102
46109
area por de

# load model 

In [ ]:
device='cpu'
model = get_instance_segmentation_model(3)
model.load_state_dict(torch.load("saved_models/generator_14.pth"))
#model.load_state_dict(torch.load("saved_models_cat_dogs_horse/generator_9.pth"))
model.to(device)
model.eval()

In [ ]:
sample_metrics = []
labels = []
model.eval()
for i,batch in enumerate(tqdm.tqdm(validation_dataloader, desc="evaluate training")):#enumerate(train_dataloader):
    images = batch[0]
    targets= batch[1]
    #labels += targets.tolist()
    
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    with torch.no_grad():
        outputs = model(images)
        precisionDF=get_precisionDF(outputs,targets)
        print(precisionDF)
    break

In [ ]:
sample_metrics = []
labels = []
model.eval()
for i,batch in enumerate(tqdm.tqdm(validation_dataloader, desc="evaluate training")):#enumerate(train_dataloader):
    images = batch[0]
    targets= batch[1]
    #labels += targets.tolist()
    
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    with torch.no_grad():
        outputs = model(images)
        precisionDF=get_precisionDF(outputs,targets)
        print(precisionDF)
    break
    
    


In [ ]:
np.mean(precisionDF['IOU'])

In [ ]:
np.sum(precisionDF['Precision'])

In [ ]:
np.mean(precisionDF['Recall'])

In [ ]:
import cv2
import matplotlib.pyplot as plt
def predict_new_image(path_to_file,model,names,threshold=0.7):
    imgoriginal=cv2.imread(path_to_file)
    im=imgoriginal.copy()
    im=im/255
    im=torch.tensor(im.transpose(2, 0, 1),dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        prediction = model([im.to(device)])
    
    boxes=prediction[0]['boxes'].to('cpu').numpy().tolist()
    scores=prediction[0]['scores'].to('cpu').numpy().tolist()
    labels=prediction[0]['labels'].to('cpu').numpy().tolist()
    
    for i in range(len(boxes)):
        copyimage=imgoriginal.copy()
        coordenadas=boxes[i]
        confianza=scores[i]
        if confianza>threshold:
            print(names[labels[i]])
            cv2.rectangle(copyimage,(int(coordenadas[0]),
                           int(coordenadas[1])),
                          (int(coordenadas[2]),
                           int(coordenadas[3])),(0,255,0),6)
        plt.imshow(cv2.cvtColor(copyimage, cv2.COLOR_BGR2RGB))
        plt.show()

In [ ]:
%matplotlib inline
path="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/images/val2017/"
imagenselected=np.random.choice(os.listdir(path), 1,replace=False)
path_test_images=path+imagenselected[0]
names=['cat','dog']
predict_new_image(path_test_images,model,names)

# training

In [ ]:
#num_epochs = 1

#for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    #train_one_epoch(model, optimizer, train_dataloader, device, epoch, print_freq=10)
    # update the learning rate
    #lr_scheduler.step()

In [ ]:
from torch.utils.tensorboard import SummaryWriter
#tensorboard --logdir=C:/Users/alejg/Documents/proyectos_personales/images_segmentation/detector/tensroboar

#tensorboard --logdir=C:/Users/alejg/Documents/tfm/image_detector_backend/train/tensroboar


In [15]:
len(train_dataloader)

2000

In [16]:
len(validation_dataloader)

2000

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
writer = SummaryWriter('tensroboar')

for epoch in tqdm.tqdm(range(epoch_ini, num_epochs), desc="Training epoch"):
    #training
    print(epoch)
    
    model.train()
    lr_scheduler2 = None
    
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(train_dataloader) - 1)
        lr_scheduler2 = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)
    
    
    for i,batch in enumerate(tqdm.tqdm(train_dataloader, desc="Training")):#enumerate(train_dataloader):
        print(f"dataloade mini batch: {i}")
        optimizer.zero_grad()
        
        images = batch[0]
        targets= batch[1]
        
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        # reduce losses over all GPUs for logging purposes
        #loss_dict_reduced = utils.reduce_dict(loss_dict)
        #losses_reduced = sum(loss for loss in loss_dict_reduced.values())
        #loss_value = losses_reduced.item()
        
        #if not math.isfinite(loss_value):
        #    print("Loss is {}, stopping training".format(loss_value))
        #    print(loss_dict_reduced)
        #    sys.exit(1)
        
        if not math.isfinite(losses):
            print("Loss is {}, stopping training".format(losses))
            print("Loss is {}, stopping training".format(loss_dict))
            print("Loss is {}, stopping training".format([loss for loss in loss_dict.values()]))
            
            
            
            sys.exit(1)
            
        print(f"la funcion de perdida vale {losses} en la iteracion {epoch}")
        #writer.add_scalar('loss', losses, epoch)
        print(losses)
        torch.cuda.empty_cache()
        
        losses.backward()
        optimizer.step()
        
        if lr_scheduler2 is not None:
                lr_scheduler2.step()
                
    lr_scheduler.step()
    #evluation in validation set
    model.eval()
    allprecisionDF = pd.DataFrame({})
    for itraineval,batch in enumerate(tqdm.tqdm(train_dataloader, desc="evaluate training set")):
        images = batch[0]
        targets= batch[1]
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
        with torch.no_grad():
            outputs = model(images)
            precisionDF=get_precisionDF(outputs,targets)
        allprecisionDF=allprecisionDF.append(precisionDF,ignore_index=True)
        
    precision=np.mean(allprecisionDF['Precision'])
    recall=np.mean(allprecisionDF['Recall'])
    IoU=np.mean(allprecisionDF['IOU'])
    writer.add_scalar('precision_training', precision, epoch)
    writer.add_scalar('recall_training', recall, epoch)
    writer.add_scalar('IoU_training', IoU, epoch)
    print(f"precesion:{precision},recall {recall},IOU {IoU}")
    #evluation in validation set
    model.eval()
    allprecisionDF = pd.DataFrame({})
    for ivalideval,batch in enumerate(tqdm.tqdm(validation_dataloader, desc="evaluate validation set")):
        images = batch[0]
        targets= batch[1]
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
        with torch.no_grad():
            outputs = model(images)
            precisionDF=get_precisionDF(outputs,targets)
        allprecisionDF=allprecisionDF.append(precisionDF,ignore_index=True)
        
    precision=np.mean(allprecisionDF['Precision'])
    recall=np.mean(allprecisionDF['Recall'])
    IoU=np.mean(allprecisionDF['IOU'])
    writer.add_scalar('precision_validation', precision, epoch)
    writer.add_scalar('recall_validation', recall, epoch)
    writer.add_scalar('IoU_validation', IoU, epoch)
    print(f"precesion:{precision},recall {recall},IOU {IoU}")
    #save pickles
    torch.save(model.state_dict(), "saved_models/generator_%d.pth" % epoch)

    

In [ ]:
for i, batch in enumerate(train_dataloader):
    if i==1:
        images = batch[0]
        targets= batch[1]
        for im in images:
            im=im/255
            im=torch.tensor(im.transpose(1, 2, 0),dtype=torch.float32)
            
        #images = list(image.to('cpu') for image in images)
        #targets = [{k: v.to('cpu') for k, v in t.items()} for t in targets]
        break

In [ ]:
im=im.to('cpu')

In [ ]:
im=im.numpy()

In [ ]:
[xmin, ymin, xmax, ymax]

In [ ]:
img=images[1].numpy()

In [ ]:
imgoriginal.shape

In [ ]:
imgO=im.copy()
im=im.reshape((imgO.shape[1], imgO.shape[2], imgO.shape[0]))

In [ ]:
imgO.shape

In [ ]:
img.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
allprecisionDF=allprecisionDF.append(pd.DataFrame({}),ignore_index=True)

In [ ]:
df

In [ ]:
def predict_new_image(imgoriginal, model, names, threshold=0.7,device='cpu'):
    im = imgoriginal.copy()
    im = im / 255
    im = torch.tensor(im.transpose(2, 0, 1), dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        prediction = model([im.to(device)])

    boxes = prediction[0]['boxes'].to('cpu').numpy().tolist()
    scores = prediction[0]['scores'].to('cpu').numpy().tolist()
    labels = prediction[0]['labels'].to('cpu').numpy().tolist()

    jsonOutput={}
    for i in range(len(boxes)):
        coordenadas = boxes[i]
        confianza = scores[i]
        if confianza > threshold:
            jsonOutput.update({str(i):{'coordinates':coordenadas,
                                       'confidence':confianza,
                                       'label':names[labels[i]]}
                               })

    return jsonOutput

In [ ]:
path_test_images="D:/datos/train2017/train2017/000000002337.jpg"
threshold=0.7
import base64
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
with open(path_test_images, "rb") as imageFile:
    file = base64.b64encode(imageFile.read())

In [ ]:
#file = base64.b64encode(image_file_read)
jpg_original = base64.b64decode(file)
jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
original_image = cv2.imdecode(jpg_as_np, flags=1)
model = get_instance_segmentation_model(2)
model.load_state_dict(torch.load('C:/Users/alejg/Documents/tfm/image_detector_backend/fastapi' + "/models/generator_13PEOPLE.pth", map_location=torch.device('cpu')))
model.to('cpu')
model.eval()
preidction=predict_new_image(original_image, model, names=['background','person'], threshold=0.7,device='cpu')

In [ ]:
preidction

In [ ]:

jpg_original = base64.b64decode(file)
jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
imgoriginal = cv2.imdecode(jpg_as_np, flags=1)

In [ ]:
im=imgoriginal.copy()
im=im/255
im=torch.tensor(im.transpose(2, 0, 1),dtype=torch.float32)
model.eval()
with torch.no_grad():
    prediction = model([im.to(device)])

boxes=prediction[0]['boxes'].to('cpu').numpy().tolist()
scores=prediction[0]['scores'].to('cpu').numpy().tolist()
for i in range(len(boxes)):
    coordenadas=boxes[i]
    confianza=scores[i]
    print(confianza)
    if confianza>threshold:
        #print(confianza)
        cv2.rectangle(imgoriginal,(int(coordenadas[0]),
                       int(coordenadas[1])),
                      (int(coordenadas[2]),
                       int(coordenadas[3])),(0,255,0),6)
plt.imshow(cv2.cvtColor(imgoriginal, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
pred=model(images, targets)

In [ ]:
pred[0]


In [ ]:
targets[0]

In [ ]:
targets[0]['boxes'].to('cpu').tolist()[0]

In [ ]:
boxes=pred[0]['boxes'].to('cpu').tolist()
scores=pred[0]['scores'].to('cpu').tolist()
for i in range(len(boxes)):
    coordenadas=boxes[i]
    confianza=scores[i]
    if confianza>0.7:
        print(coordenadas)

In [ ]:
%tb